<a href="https://colab.research.google.com/github/isegura/TextClassification/blob/master/Exercise_SpamDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing several classifiers for spam detection

The goal of this exercise is to compare different classifiers for the taks of spam detection. Some possible classifiers are:

1. Logistic Regression
2. Linear Support Vector Machine
3. Random Forest
4. Multi-layer Perceptron classifier.
5. k-Nearest Neighbors classifier
6. Naïve Bayes. 

You will have to train an test these models using the spam detection dataset. 
Compare their results and propose the best classifier. 


Dataset:  https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

## Loading data




In [1]:
from google.colab import drive
drive.mount("/content/drive/")
sst_home='drive/My Drive/Colab Notebooks/'
#modify this path 
path=sst_home+'TESI/3-TextClasification/data/SMSSpamCollection'

import pandas
import csv

messages = pandas.read_csv(path, sep='\t', quoting=csv.QUOTE_NONE, names=["label", "message"])


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Cleaning texts
The following cell contains the code to clean the texts. 

In [5]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
stopwords_en = stopwords.words("english")

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re


def cleanText(text):
    text=str(text).lower()
    #tokeniza the text
    tokens=word_tokenize(text)
    #remove the stopwords
    tokens = [word for word in tokens if word not in stopwords_en]
    #(4) obtain the stems
    tokens = [PorterStemmer().stem(word) for word in tokens]
    #(5) finally, remove words with len <3 and words that contain numbers, puntuaction, ect
    min_length = 3
    p = re.compile('[a-zA-Z]+');
    filtered_tokens=[]
    for token in tokens:
        if len(token)>=min_length and p.match(token):
            filtered_tokens.append(token)
            
    return filtered_tokens



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Split the dataset: training and test datasets



In [9]:
from sklearn.model_selection import cross_val_score, train_test_split 

msg_train, msg_test, label_train, label_test = train_test_split(messages['message'], messages['label'], test_size=0.2)
print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))

4459 1115 5574


## Using TF-IDF model to represent texts 
Now, we are going to train a tf-idf model using the training dataset. Then, we will apply this tf-idf model to tranform the training and test sets. Remember that first you need to obtain the bow model.


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

bow = CountVectorizer(analyzer=cleanText).fit(msg_train)
#transform the training set to bow model
bow_train = bow.transform(msg_train)
#transform the test set to bow model
bow_test=bow.transform(msg_test)


#learn the tf-idf model from the training bow
tfidf_transformer = TfidfTransformer().fit(bow_train)
#transform the training set to tf-idf model
tfidf_train = tfidf_transformer.transform(bow_train)
#transform the test set to tf-idf model
tfidf_test = tfidf_transformer.transform(bow_test)

print('matrices obtained')

matrices obtained


## Training and test a classifier

The following cell shows you how to train and test a classifier. In particular, we use the **MultinomialNB** class, which implements the **Naive Bayes** classifier.

In [16]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=cleanText)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

pipeline.fit(msg_train,label_train)
predictions = pipeline.predict(msg_test)

print( classification_report(label_test, predictions))


              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       961
        spam       0.97      0.75      0.85       154

    accuracy                           0.96      1115
   macro avg       0.97      0.88      0.91      1115
weighted avg       0.96      0.96      0.96      1115



Now, you should add the code to train the models using other classifiers. Compare their results and choose the best. 